In [1]:
import batch

In [2]:
import pandas as pd
from datetime import datetime

In [3]:
def dt(hour, minute, second=0):
    return datetime(2023, 1, 1, hour, minute, second)

In [4]:
data = [
        (None, None, dt(1, 1), dt(1, 10)),
        (1, 1, dt(1, 2), dt(1, 10)),
        (1, None, dt(1, 2, 0), dt(1, 2, 59)),
        (3, 4, dt(1, 2, 0), dt(2, 2, 1)),      
    ]

In [5]:
columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']
df = pd.DataFrame(data, columns=columns)
df

,PULocationID,DOLocationID,tpep_pickup_datetime,tpep_dropoff_datetime
0,NaN,NaN,2023-01-01 01:01:00,2023-01-01 01:10:00
1,1.0,1.0,2023-01-01 01:02:00,2023-01-01 01:10:00
2,1.0,NaN,2023-01-01 01:02:00,2023-01-01 01:02:59
3,3.0,4.0,2023-01-01 01:02:00,2023-01-01 02:02:01


In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [7]:
df

,PULocationID,DOLocationID,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,NaN,NaN,2023-01-01 01:01:00,2023-01-01 01:10:00,9.000000
1,1.0,1.0,2023-01-01 01:02:00,2023-01-01 01:10:00,8.000000
2,1.0,NaN,2023-01-01 01:02:00,2023-01-01 01:02:59,0.983333
3,3.0,4.0,2023-01-01 01:02:00,2023-01-01 02:02:01,60.016667


In [8]:
categorical = ['PULocationID', 'DOLocationID']
actual_dataframe = batch.prepare_data(df, categorical)
actual_dataframe

,PULocationID,DOLocationID,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,-1,-1,2023-01-01 01:01:00,2023-01-01 01:10:00,9.0
1,1,1,2023-01-01 01:02:00,2023-01-01 01:10:00,8.0


In [13]:
actual_dataframe.dtypes

PULocationID                     object
DOLocationID                     object
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
duration                        float64
dtype: object

In [9]:
expected_columns = columns + ['duration']
expected_columns

['PULocationID',
 'DOLocationID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'duration']

In [20]:
expected_data = [
        ('-1', '-1', dt(1, 1), dt(1, 10), 9.0),
        ('1', '1', dt(1, 2), dt(1, 10), 9.0),
]
expected_dataframe = pd.DataFrame(expected_data, columns=expected_columns)
expected_dataframe

,PULocationID,DOLocationID,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,-1,-1,2023-01-01 01:01:00,2023-01-01 01:10:00,9.0
1,1,1,2023-01-01 01:02:00,2023-01-01 01:10:00,9.0


In [21]:
expected_dataframe.dtypes

PULocationID                     object
DOLocationID                     object
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
duration                        float64
dtype: object

In [22]:
from pandas.testing import assert_frame_equal

In [23]:
assert_frame_equal(actual_dataframe, expected_dataframe)

AssertionError: DataFrame.iloc[:, 4] (column name="duration") are different

DataFrame.iloc[:, 4] (column name="duration") values are different (50.0 %)
[index]: [0, 1]
[left]:  [9.0, 8.0]
[right]: [9.0, 9.0]
At positional index 1, first diff: 8.0 != 9.0

In [24]:
df_result = actual_dataframe == expected_dataframe
df_result

,PULocationID,DOLocationID,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,True,True,True,True,True
1,True,True,True,True,False


In [25]:
df_result.min().sum()

4